In [1]:
using QuantumACES, LinearAlgebra, SparseArrays, StatsBase, LsqFit
using Plots, Plots.PlotMeasures, StatsPlots, ColorSchemes, LaTeXStrings
using PrettyTables, Suppressor, JLD2
# Set default plot parameters
@suppress pgfplotsx()
default(grid=false, fillalpha=0.5, tick_direction=:out, margins=0px,
    titlefontsize=10, guidefontsize=9, tickfontsize=7, legendfontsize=7,
    legendfonthalign=:left)
push!(PGFPlotsX.CUSTOM_PREAMBLE, "\\usepackage{amsmath}")
push!(PGFPlotsX.CUSTOM_PREAMBLE, "\\usepackage{amssymb}")
# :tol_bright is a colourblind-friendly colourscheme
tol_bright = ColorSchemes.tol_bright
colour_series = [tol_bright[3]; tol_bright[1]; tol_bright[2]; tol_bright[4]; tol_bright[5]]
# In REVTeX, one column width is 246 pt, and the text width is 510 pt
col_width = 246.0
text_width = 510.0
aspect_ratio = 4 / 3
inches_per_pt = 1.0 / 72.27
default_dpi = 100.0
col_width_px = col_width * inches_per_pt * default_dpi
text_width_px = text_width * inches_per_pt * default_dpi
col_height_px = col_width_px / aspect_ratio
text_height_px = text_width_px / aspect_ratio
triple_height = (1 + 2 * (text_width / col_width - 1)) * col_width / aspect_ratio
triple_height_px = triple_height * inches_per_pt * default_dpi;

In [2]:
# Load the data
big_decoding_dict = load("data/big_decoding_data.jld2")
big_memory_data = big_decoding_dict["big_memory_data"]
big_memory_summary = big_decoding_dict["big_memory_summary"]
rounds = 25
big_shots = big_memory_summary.shots
memory_summary_rounds = big_decoding_dict["memory_summary_rounds"]
rounds_set = [3; 5; 9; 17; 33]
rounds_num = length(rounds_set)
decoder_labels = ["True noise"; "Tuned depolarising"; "ACES: 10^6 shots"; "ACES: 10^7 shots"]
decoder_num = length(decoder_labels)
shots = memory_summary_rounds[1].shots
# Check the data
@assert rounds == big_memory_summary.rounds
for round_idx in 1:rounds_num
    memory_summary = memory_summary_rounds[round_idx]
    @assert shots == memory_summary.shots
    @assert decoder_num == length(memory_summary.decoder_labels)
    @assert rounds_set[round_idx] == memory_summary.rounds
end
display(big_memory_summary)

┌──────────────────┬─────────────────────┬─────────────────────┐
│ Decoder label    │ Memory X error rate │ Memory Z error rate │
├──────────────────┼─────────────────────┼─────────────────────┤
│ True             │ (0.0512 ± 0.0011)%  │ (0.0589 ± 0.001)%   │
│ Depolarising     │ (0.0674 ± 0.0012)%  │ (0.0765 ± 0.0012)%  │
│ ACES S'=1000000  │ (0.0528 ± 0.0011)%  │ (0.0599 ± 0.001)%   │
│ ACES S'=10000000 │ (0.0516 ± 0.0011)%  │ (0.0592 ± 0.001)%   │
└──────────────────┴─────────────────────┴─────────────────────┘


In [3]:
# Display the confusion matrix
decoder_x_confusion = big_memory_summary.decoder_x_confusion
decoder_z_confusion = big_memory_summary.decoder_z_confusion
decoder_confusion = decoder_x_confusion + decoder_z_confusion
decoder_order = [1; 4; 3; 2]
decoder_confusion_ordered = decoder_confusion[decoder_order, decoder_order]
pretty_table(hcat(decoder_labels[decoder_order], decoder_confusion_ordered),
    header=["Succ. \\ Fail."; decoder_labels[decoder_order]],
    alignment=:l,
)

┌────────────────────┬────────────┬──────────────────┬──────────────────┬────────────────────┐
│ Succ. \\ Fail.     │ True noise │ ACES: 10^7 shots │ ACES: 10^6 shots │ Tuned depolarising │
├────────────────────┼────────────┼──────────────────┼──────────────────┼────────────────────┤
│ True noise         │ 5507       │ 227              │ 619              │ 3005               │
│ ACES: 10^7 shots   │ 195        │ 5539             │ 564              │ 2997               │
│ ACES: 10^6 shots   │ 495        │ 472              │ 5631             │ 2994               │
│ Tuned depolarising │ 1314       │ 1338             │ 1427             │ 7198               │
└────────────────────┴────────────┴──────────────────┴──────────────────┴────────────────────┘


In [4]:
# Print the logical error ratios
memory_errors = diag(decoder_confusion)
pretty_table(hcat(decoder_labels, memory_errors / memory_errors[1]),
    header=["Decoder"; "Distance 25 relative LER for 25 rounds"],
    alignment=:l,
)

┌────────────────────┬────────────────────────────────────────┐
│ Decoder            │ Distance 25 relative LER for 25 rounds │
├────────────────────┼────────────────────────────────────────┤
│ True noise         │ 1.0                                    │
│ Tuned depolarising │ 1.30706                                │
│ ACES: 10^6 shots   │ 1.02252                                │
│ ACES: 10^7 shots   │ 1.00581                                │
└────────────────────┴────────────────────────────────────────┘


In [5]:
# Fit the memory errors
rounds_x_errors = Vector{Float64}(undef, decoder_num)
rounds_x_errors_se = Vector{Float64}(undef, decoder_num)
rounds_z_errors = Vector{Float64}(undef, decoder_num)
rounds_z_errors_se = Vector{Float64}(undef, decoder_num)
for decoder_idx in 1:decoder_num
    # Memory X error
    memory_x_errors = [memory_summary.memory_x_errors[decoder_idx] for memory_summary in memory_summary_rounds]
    memory_x_errors_se = [sqrt(memory_summary.memory_x_errors_cov[decoder_idx, decoder_idx] / memory_summary.shots) for memory_summary in memory_summary_rounds]
    (round_x_params, round_x_params_cov) = fit_round_error(rounds_set, memory_x_errors, memory_x_errors_se; return_cov=true)
    (round_x_error, round_x_error_se) = get_round_error(round_x_params, round_x_params_cov)
    rounds_x_errors[decoder_idx] = round_x_error
    rounds_x_errors_se[decoder_idx] = round_x_error_se
    # Memory Z error
    memory_z_errors = [memory_summary.memory_z_errors[decoder_idx] for memory_summary in memory_summary_rounds]
    memory_z_errors_se = [sqrt(memory_summary.memory_z_errors_cov[decoder_idx, decoder_idx] / memory_summary.shots) for memory_summary in memory_summary_rounds]
    (round_z_params, round_z_params_cov) = fit_round_error(rounds_set, memory_z_errors, memory_z_errors_se; return_cov=true)
    (round_z_error, round_z_error_se) = get_round_error(round_z_params, round_z_params_cov)
    rounds_z_errors[decoder_idx] = round_z_error
    rounds_z_errors_se[decoder_idx] = round_z_error_se
end

In [6]:
# Average the memory errors
rounds_errors = (rounds_x_errors + rounds_z_errors) / 2
rounds_errors_se = sqrt.(rounds_x_errors_se .^ 2 + rounds_z_errors_se .^ 2) / 2
# Print the memory errors
scale_factor = 5
digits = 3
rounds_errors_string = [
    "($(round(10^scale_factor * rounds_errors[j], digits = digits)) ± $(round(10^scale_factor * rounds_errors_se[j], digits = digits))) x 1e$(-scale_factor)"
    for j in 1:decoder_num
]
pretty_table(hcat(decoder_labels, rounds_errors_string),
    header=["Decoder"; "Distance 25 LER per round"],
    alignment=:l,
)

┌────────────────────┬───────────────────────────┐
│ Decoder            │ Distance 25 LER per round │
├────────────────────┼───────────────────────────┤
│ True noise         │ (2.392 ± 0.05) x 1e-5     │
│ Tuned depolarising │ (3.126 ± 0.058) x 1e-5    │
│ ACES: 10^6 shots   │ (2.421 ± 0.05) x 1e-5     │
│ ACES: 10^7 shots   │ (2.4 ± 0.05) x 1e-5       │
└────────────────────┴───────────────────────────┘


In [7]:
# Print the relative memory errors
rounds_relative_errors = rounds_errors ./ rounds_errors[1]
# This currently does not account for covariance information
# In truth the standard errors will be smaller
rounds_relative_errors_se = rounds_relative_errors .* sqrt.(rounds_errors_se .^ 2 ./ rounds_errors .^ 2 .+ rounds_errors_se[1] .^ 2 / rounds_errors[1] .^ 2)
digits = 4
rounds_relative_errors_string = [
    "$(round(rounds_relative_errors[j], digits = digits)) ± $(round( rounds_relative_errors_se[j], digits = digits))"
    for j in 1:decoder_num
]
pretty_table(hcat(decoder_labels, rounds_relative_errors_string),
    header=["Decoder"; "Distance 25 relative LER per round"],
    alignment=:l,
)

┌────────────────────┬────────────────────────────────────┐
│ Decoder            │ Distance 25 relative LER per round │
├────────────────────┼────────────────────────────────────┤
│ True noise         │ 1.0 ± 0.0297                       │
│ Tuned depolarising │ 1.3068 ± 0.0367                    │
│ ACES: 10^6 shots   │ 1.012 ± 0.0299                     │
│ ACES: 10^7 shots   │ 1.0031 ± 0.0297                    │
└────────────────────┴────────────────────────────────────┘
